In [ ]:
from pathlib import Path
import webbrowser

import pandas as pd
import matplotlib.pyplot as plt
import yaml

from evolution.evaluation.evaluator import EnROADSEvaluator
from evolution.utils import process_config
from experiments.experiment_utils import NNExperimenter, DirectExperimenter
from enroadspy import load_input_specs, id_to_name, name_to_id
from enroadspy.generate_url import actions_to_url

In [ ]:
results_dir = Path("results/actions")
results_df = pd.read_csv(results_dir / "results.csv")
n_generations = results_df["gen"].max()
with open(results_dir / "config.yml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

config = process_config(config)

context = config["context"]
actions = config["actions"]
outcomes = config["outcomes"]
outcome_keys = list(outcomes.keys())
n_elites = config["evolution_params"]["n_elites"]
print(n_generations, outcomes, len(actions), len(context))

In [ ]:
input_specs = load_input_specs()

evaluator = EnROADSEvaluator(context, actions, outcomes, n_jobs=1, batch_size=config["batch_size"], device=config["device"])
if len(context) > 0:
    experimenter = NNExperimenter(results_dir)
else:
    experimenter = DirectExperimenter(results_dir)

In [ ]:
def filter_actions_dict(actions_dict: dict):
    input_specs = load_input_specs()
    filtered = {}
    for action in actions_dict:
        default = input_specs[input_specs["id"] == action]["defaultValue"].values[0]
        if actions_dict[action] != default:
            filtered[action] = actions_dict[action]
    return filtered

In [ ]:
def plot_actions_taken(gen: int, results_df: pd.DataFrame, magnitude=False):
    gen_df = results_df[results_df["gen"] == gen]
    gen_df = gen_df[gen_df["rank"] == 1]
    pareto_len = len(gen_df)
    actions_count = {}
    for cand_id in gen_df["cand_id"]:
        [context_actions_dict], _, _ = experimenter.get_candidate_results(cand_id)
        for action in context_actions_dict:
            row = input_specs[input_specs["id"] == action].iloc[0]
            default_value = row["defaultValue"]
            if action not in context and context_actions_dict[action] != default_value:
                if not magnitude:
                    actions_count[action] = actions_count.get(action, 0) + 1
                else:
                    min_value = row["minValue"] if row["kind"] == "slider" else row["offValue"]
                    max_value = row["maxValue"] if row["kind"] == "slider" else row["onValue"]
                    if min_value == max_value:
                        continue
                    norm = (context_actions_dict[action] - min_value) / (max_value - min_value)
                    default_norm = (default_value - min_value) / (max_value - min_value)
                    actions_count[action] += abs(norm - default_norm)

    # Switch to nice name, normalize values
    actions_count = {action: count / pareto_len for action, count in actions_count.items()}
    actions_count = dict(sorted(actions_count.items(), key=lambda item: item[1], reverse=True))
    pretty_actions_count = {id_to_name(action, input_specs): count for action, count in actions_count.items()}

    print(pretty_actions_count)
    fig, ax = plt.subplots(figsize=(5, 20))
    ax.barh(pretty_actions_count.keys(), pretty_actions_count.values())
    ax.set_title("Proportion of Candidates in Pareto with Action Taken")
    ax.grid()
    ax.invert_yaxis()
    plt.show()

    return actions_count

actions_count = plot_actions_taken(n_generations, results_df, magnitude=False)

In [ ]:
def plot_pareto(outcome1: str, outcome2: str, gen: int, results_df: pd.DataFrame, colored_ids=[]):

    gen_df = results_df[(results_df["gen"] == gen) & (results_df["rank"] == 1)]

    plt.scatter(gen_df[outcome1], gen_df[outcome2])

    colors = ["pink", "lightgreen"]
    for colored_id, color in zip(colored_ids, colors):
        colored_df = gen_df[gen_df["cand_id"] == colored_id]
        plt.scatter(colored_df[outcome1], colored_df[outcome2], color=color)

    title = f"{outcome1} vs {outcome2} Final Pareto"
    plt.title(title)
    plt.ylabel(outcome2)
    plt.xlabel(outcome1)
    plt.show()

plot_pareto(outcome_keys[0], outcome_keys[2], n_generations, results_df)
plot_pareto(outcome_keys[1], outcome_keys[2], n_generations, results_df)

In [ ]:
def get_special_cands(outcome: str, results_df: pd.DataFrame, gen: int):
    """
    Sort by largest difference in outcome. Then get the 2 candidates with the largest difference.
    """
    pareto_df = results_df[(results_df["gen"] == gen) & (results_df["rank"] == 1)]
    pareto_df = pareto_df.sort_values(outcome, ascending=False)
    diff = pareto_df[outcome].diff()
    diff = diff.fillna(0)

    # Get the index of the row where the largest drop occurs (this is the lower value of the pair)
    top_idx = diff.idxmax()

    # Get the position (iloc) of that row to access the previous row
    top_pos = pareto_df.index.get_loc(top_idx)

    # if lower_pos == 0:
    #     raise ValueError("Largest diff is at the top row; cannot get the row above it.")

    bot_pos = top_pos + 1
    bot_idx = pareto_df.index[bot_pos]

    top = pareto_df.loc[top_idx]
    bot = pareto_df.loc[bot_idx]

    return top["cand_id"], bot["cand_id"]


In [ ]:
special = get_special_cands(outcome_keys[2], results_df, n_generations)
print(special)
plot_pareto(outcome_keys[0], outcome_keys[2], n_generations, results_df, special)
plot_pareto(outcome_keys[1], outcome_keys[2], n_generations, results_df, special)

In [ ]:
def compare_actions(cand_1: str, cand_2: str):
    [ca_1], _, _ = experimenter.get_candidate_results(cand_1)
    [ca_2], _, _ = experimenter.get_candidate_results(cand_2)

    total_actions = list(set(ca_1.keys()).union(set(ca_2.keys())))

    all_actions = []
    norm1s = []
    norm2s = []
    defaults = []
    mins = []
    maxes = []

    for action in total_actions:
        row = input_specs[input_specs["id"] == action].iloc[0]
        default_value = row["defaultValue"]
        min_value = row["minValue"] if row["kind"] == "slider" else row["offValue"]
        max_value = row["maxValue"] if row["kind"] == "slider" else row["onValue"]

        if ca_1[action] == default_value and ca_2[action] == default_value:
            continue

        if min_value == max_value:
            norm_1 = 1
            norm_2 = 1
            norm_default = 1

        else:
            norm_default = (default_value - min_value) / (max_value - min_value)
            norm_1 = (ca_1[action] - min_value) / (max_value - min_value)
            norm_2 = (ca_2[action] - min_value) / (max_value - min_value)

            diff_1 = norm_1 - norm_default
            diff_2 = norm_2 - norm_default

        norm1s.append(diff_1)
        norm2s.append(diff_2)
        defaults.append(norm_default)
        mins.append(min_value)
        maxes.append(max_value)
        all_actions.append(action)

    df = pd.DataFrame({"action": all_actions,
                       "norm1": norm1s,
                       "norm2": norm2s,
                       "default": defaults,
                       "min": mins,
                       "max": maxes})
    
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.scatter(df["norm1"], range(len(df)), label=cand_1, color="pink")
    ax.scatter(df["norm2"], range(len(df)), label=cand_2, color="lightgreen")
    ax.axvline(x=0, color="black", linestyle="--", label="default")

    ax.set_xlabel("Normalized Difference from Default")
    ax.set_yticks(range(len(df)), df["action"].apply(lambda x: id_to_name(x, input_specs)))
    ax.set_title("Actions Comparison")
    ax.grid()
    ax.legend()

    plt.show()

compare_actions(*special)

In [ ]:
def print_actions_dict(cand_id):
    [context_actions_dict], [outcomes_df], _ = experimenter.get_candidate_results(cand_id)
    true_ca_dict = {}
    for action in context_actions_dict:
        default_value = input_specs[input_specs["id"] == action]["defaultValue"].values[0]
        if context_actions_dict[action] != default_value:
            true_ca_dict[action] = context_actions_dict[action]
    print(true_ca_dict)

In [ ]:
for cand_id in special:
    print_actions_dict(cand_id)

In [ ]:
def color_pareto_by_action(action: str, outcome_1: str, outcome_2: str, gen: int, results_df: pd.DataFrame):
    pareto_df = results_df[(results_df["gen"] == gen) & (results_df["rank"] == 1)]

    action = name_to_id(action, input_specs)

    action_vals = []
    for cand_id in pareto_df["cand_id"]:
        [context_actions_dict], _, _ = experimenter.get_candidate_results(cand_id)
        action_vals.append(context_actions_dict[action])
    
    pareto_df["action"] = action_vals

    fig, ax = plt.subplots()
    min_val = min(action_vals)
    max_val = max(action_vals)
    if min_val < 0 and max_val > 0:
        cmap_params = {"cmap": "PiYG", "vmin": min(min_val, -max_val), "vmax": max(-min_val, max_val)}
    else:
        cmap_params = {"cmap": "magma"}
    scatter = ax.scatter(pareto_df[outcome_1], pareto_df[outcome_2], c=pareto_df["action"], **cmap_params)
    ax.set_xlabel(outcome_1)
    ax.set_ylabel(outcome_2)
    cbar = fig.colorbar(scatter)
    cbar.set_label(id_to_name(action, load_input_specs()), rotation=270, labelpad=15)
    plt.show()

color_pareto_by_action("Carbon tax initial target", outcome_keys[0], outcome_keys[2], n_generations, results_df)
color_pareto_by_action("Source tax oil boe", outcome_keys[0], outcome_keys[2], n_generations, results_df)

In [ ]:
pareto_df = results_df[(results_df["gen"] == n_generations) & (results_df["rank"] == 1)]
[context_actions_dict], _, _ = experimenter.get_candidate_results(pareto_df.sort_values(outcome_keys[2])["cand_id"].values[0])
print(filter_actions_dict(context_actions_dict))
if config["decomplexify"]:
    context_actions_dict = evaluator.decomplexify_actions_dict(context_actions_dict)
url = actions_to_url(context_actions_dict)
webbrowser.open(url)

In [ ]:
color_pareto_by_action("Carbon tax final target", outcome_keys[0], outcome_keys[2], n_generations, results_df)